In [2]:
import os 
from dotenv import load_dotenv

# load_dotenv("../../.env")

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]="HR_BOT"
os.environ["LANGSMITH_ENDPOINT"]=os.getenv("LANGSMITH_ENDPOINT")

In [3]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.messages import HumanMessage,SystemMessage,ToolMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_chroma import Chroma 
from langchain.chains.combine_documents import create_stuff_documents_chain  
from langchain_openai import ChatOpenAI

In [4]:
llm=ChatOpenAI(model="gpt-5")

In [5]:
embedding=OpenAIEmbeddings(model="text-embedding-3-large")

/var/folders/dd/ffn14k7x4bz7kmmd0l7kmqg80000gn/T/ipykernel_12200/2347312799.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings(model="text-embedding-3-large")


In [6]:
db2=Chroma(persist_directory="../vector_data/hr_csv",embedding_function=embedding)

In [7]:
retriever=db2.as_retriever()

In [12]:
prompt=ChatPromptTemplate.from_template(
    """
    <user_input>
    {input}
    </user_input>

    You are a helpful assistant for HR in finsolve company . Your name is FinSolve HR.
    - answer question of user using below context 
    - answer qestion structure format 
    - provide answer asked by user, if not found, say "I don't know".
    
    <context>
    {context}
    </context>
    """
)

In [9]:
chain=create_stuff_documents_chain(llm,prompt)

In [13]:
user="give me employee_id of Aadhya Patel in Sales department ? "

result=chain.invoke({"input":user,
                     "context":retriever.get_relevant_documents(user)
})

In [14]:
print(result)

employee_id: FINEMP1000


In [15]:
chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    <user_input>\n    {input}\n    </user_input>\n\n    You are a helpful assistant for HR in finsolve company . Your name is FinSolve HR.\n    - answer question of user using below context \n    - answer qestion structure format \n\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12dcff020>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12e9abda0>, root_client=<openai.OpenAI object at 0x12d9bb470>, root_asy